In [ ]:
!pip install pandas matplotlib
!pip install -U kaleido


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go

# Define the file path for your data
file_path = '/content/Task11_chemicals_gene.csv'  # Update this with your CSV file path

# Read the CSV file into a dataframe
df = pd.read_csv(file_path)

# Create an undirected graph
G = nx.Graph()

# Add edges to the graph
for _, row in df.iterrows():
    G.add_edge(row['Chemicals'], row['interaction_type'], weight=row.get('Weight', 1))

# Create a layout for nodes
pos = nx.spring_layout(G)  # You can use different layouts like 'circular_layout', 'kamada_kaway_layout', etc.

# Extract node positions for plotting
node_x = []
node_y = []
node_text = []

for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)

# Extract edge positions for plotting
edge_x = []
edge_y = []
edge_text = []

for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_text.append(edge[2].get('weight', 1))

# Create 2D plot
fig = go.Figure()

# Add edges
fig.add_trace(go.Scatter(
    x=edge_x,
    y=edge_y,
    mode='lines',
    line=dict(width=1, color='black'),
    hoverinfo='text',
    text=edge_text,
    name='Edges'
))

# Add nodes
fig.add_trace(go.Scatter(
    x=node_x,
    y=node_y,
    mode='markers+text',
    marker=dict(size=10, color='blue'),
    text=node_text,
    textposition='top center',
    name='Nodes'
))

# Set layout
fig.update_layout(
    title='Graph Visualization',
    showlegend=True,
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False)
)

# Save as HTML file
fig.write_html('/content/graph_visualization.html')

# Show the plot (optional)
fig.show()


Graph to Knowledge Graph


In [ ]:
!pip install pandas networkx plotly rdflib



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.9 MB/s eta 0:00:00


In [ ]:
from urllib.parse import quote

In [ ]:
pip install dash dash-cytoscape


In [ ]:
import pandas as pd
import re
import networkx as nx
import plotly.graph_objects as go
from rdflib import Graph, URIRef, Namespace
from plotly.subplots import make_subplots

# Define the file path for your RDF data
ttl_file_path = '/content/knowledge_graph.ttl'

# Load the RDF graph from the Turtle file
rdf_graph = Graph()
rdf_graph.parse(ttl_file_path, format='turtle')

# Create a directed graph
G = nx.DiGraph()

# Define Namespaces
EX = Namespace("http://example.org/")
CHEM = Namespace("http://example.org/chemical/")
GENE = Namespace("http://example.org/gene/")
INTER = Namespace("http://example.org/interaction/")

# Extract triples from the RDF graph and add to network graph
for s, p, o in rdf_graph:
    s_str = s.split('/')[-1]
    o_str = o.split('/')[-1]

    if isinstance(s, URIRef) and isinstance(o, URIRef):
        if p == EX.interactsWith:
            G.add_edge(s_str, o_str, interaction_type=str(p))
        elif p == EX.hasInteractionType:
            if not G.has_node(s_str):
                G.add_node(s_str)
            G.nodes[s_str]['interaction_type'] = str(o)

# Create a layout for nodes
pos = nx.spring_layout(G, dim=2)

# Extract node positions for plotting
node_x = []
node_y = []
node_text = []

for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)

# Extract edge positions for plotting
edge_x = []
edge_y = []
edge_text = []

for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_text.append(edge[2].get('interaction_type', ''))

# Create 2D plot with interactivity
fig = go.Figure()

# Add edges
fig.add_trace(go.Scatter(
    x=edge_x,
    y=edge_y,
    mode='lines',
    line=dict(width=1, color='black'),
    hoverinfo='text',
    text=edge_text,
    name='Edges'
))

# Add nodes
fig.add_trace(go.Scatter(
    x=node_x,
    y=node_y,
    mode='markers+text',
    marker=dict(size=10, color='blue'),
    text=node_text,
    textposition='top center',
    name='Nodes'
))

# Set layout with interactivity
fig.update_layout(
    title='Interactive RDF Knowledge Graph',
    showlegend=True,
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    clickmode='event+select'
)

# Add interactive capabilities
fig.update_traces(marker=dict(size=15),
                  selector=dict(mode='markers+text'))

# Save as HTML file
fig.write_html('/content/interactive_graph_visualization.html')

# Show the plot (optional)
fig.show()
